# TODOs
- [X] send email to Errol - does the team that installed an asset do all other maintenance related events for the same asset? Useful for determining team productivity and effectiveness.
- [X] Meaning of previous_repairs and previous_unplanned in assets table.
At first it seemed that it was simply the values of the last row from the events table, but it doesn't seem to match up.
- [ ] Meaning of non-zero previous_repairs and previous_unplanned in first event after installation.
- [X] Transform datetimes and time periods to numeric data
- [X] Augment assets data with events statistics: number of replacements, number of repairs, average time between events, ...?
- [ ] Save the samples that are not included for the model training and testing (from the down sampling) and use them for testing. Will make a very large test set.


`01MAI`
- Trained the dataset with a Random Forest Classifier on the horizon 30 class. Very high accuracy.
- Looked at the class distribution and observed that it was very imbalanced.
- Downsamples to have a 50/50 split.
- Retrained. Very high accuracy.


Source:
- Predicteve maintenance
  - [predictive maintenance article - towardsdatascience](https://towardsdatascience.com/how-to-implement-machine-learning-for-predictive-maintenance-4633cdbe4860)
- Imbalanced data
  - [dealing with imbalanced class distribution](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)
  - [what is imbalanced data](https://machinelearningmastery.com/what-is-imbalanced-classification/)

In [ ]:
# staDecisionTreeClassifierd library
from typing import List, Union, Optional, Tuple
import datetime
from collections import namedtuple

# data and viz
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# numeric
import math
import numpy as np

# data prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# models
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression


# metrics
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import tree

# prod
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

# 1. Data exploration

## 1.1. Load data

In [ ]:
datafiles = !ls data/*

In [ ]:
datasets = {}
for fn in datafiles:
    dataset_name = fn.split('/')[-1].rstrip('.csv')
    datasets[dataset_name] = pd.read_csv(fn)

for name, dataset in datasets.items():
    print('\n'*3, name, '\n')
    print(dataset)

In [ ]:
(datasets['replacement_data'].shape[0] + datasets['repair_data'].shape[0]) == datasets['planned_data'].shape[0]

The planned data seems to contain all the repair and replacement events.

## 1.2. Join all datasets in 2 tables (events, assets)

We seem to have 2 types of data. We'll start the process of combining all tables into 2 tables for each of these types:
- time series data for maintenance events ("events");
- and assets' attributes data ("assets").



### 1.2.1. Join events

To combine these tables, we will:
1. We'll need to first add a column to repair_data and replacement_data to indicate the type of event.
2. All columns are the same, so we can concatenate repair and replacement events.
3. Inner join events with planned data. We just need the planned column.

Let's call this new dataset "events".

In [ ]:
datasets['replacement_data']['type'] = 'replacement'
datasets['repair_data']['type'] = 'repair'

In [ ]:
datasets['all_events_data'] = pd.concat([datasets['replacement_data'], datasets['repair_data']])

In [ ]:
events = pd.merge(datasets['all_events_data'], datasets['planned_data'], how='inner', on=['event_id', 'asset_id', 'event_date'])

In [ ]:
date_cols = ['event_date', 'installed_date']
for col in date_cols:
    events[col] = pd.to_datetime(events[col])

In [ ]:
events.sort_values(by=['asset_id', 'event_date'])

### 1.2.2 Join assets

Let's join the attributes of the assets in a single table:
- asset_attribute_data_general
- asset_attribute_data_usage
- asset_attribute_data_weather
- asset_data

Let's call this new dataset "assets".

In [ ]:
assets = pd.merge(datasets['asset_attribute_data_general'], datasets['asset_attribute_data_usage'], on='asset_id')
assets = pd.merge(assets, datasets['asset_attribute_data_weather'], on='asset_id')
assets = pd.merge(assets, datasets['asset_data'], on='asset_id')

In [ ]:
date_cols = ['end_date', 'start_date']
for col in date_cols:
    assets[col] = pd.to_datetime(assets[col])

In [ ]:
assets['total_useful_life'] = assets['end_date'] - assets['start_date']

In [ ]:
assets = assets.rename(columns={'previous_repairs': 'start_previous_repairs',
               'previous_unplanned': 'start_previous_unplanned'})

## 1.3. Check for gaps in datasets

Now we have only 2 tables to work with. One refers to data about maintenance events, the other about asset attributes.

Let's check if there are gaps in the data:
- Do all assets have maintenance events? If not, why?
- Are there events refering to missing assets? These might need to be discarded depending on the following analysis.

In [ ]:
# do all assets have maintenance events?
assets_that_broke = events['asset_id'].unique()
print(f'{len(assets_that_broke)}\t assets that have replacement or repair events')

print(f'{assets.shape[0]}\t assets')

assets_without_events = assets[assets['asset_id'].isin(assets_that_broke) == False]
print(f'{assets_without_events.shape[0]}\t assets without events')

In [ ]:
assets_without_events

In [ ]:
assets.describe()

Of the 200 registered assets, we have maintenance events on 194.
Looking at the data from the 6 that didn't have incidents, no pattern is identified about their attributes.
Different teams installed them, they have different materials, locations, weather and were operational on different years.
The only similarity is that all these assets have a total useful life well below the 25% percentile.
However, there are assets that had a shorter useful life and still had maintenance events.

Let's check the statistics for time between events to decide whether to consider that these 6 assets are outliers and exclude them from further exploration.

installed_date is the date of the last intervention, either installation, which matches the start_date in the assets table, or the last repair or replacement.

In [ ]:
events['time_since_last_event'] = events['event_date'] - events['installed_date']

In [ ]:
events['time_since_last_event'].describe()

In [ ]:
q = 0.9
print(events['time_since_last_event'].quantile(q))
print(f'# events over quantile {q}: {2032*(1-q)}')

For 10% of events (~203 events), the time elapsed since the previous event was higher than 321 days.
Of the 6 assets than didn't have events, 5 have a useful life below 300 days.
This means that it's plausible that these 6 assets didn't have any maintenance events, given their brief useful life, and we'll reject the hypothesis that it's due to missing data in the events table


They will not be removed from the analysis when considering only assets' attributes.

## 1.4. Data Strategy

The events and assets tables are two different kinds of data and we can use them to answer lots of questions.
1. Which asset attributes are correlated with number of maintenance events?
2. Which attributes are correlated with total useful life?
3. Are there better performing teams?
4. Should we be avoiding certain materials in specific locations or weather clusters?
5. Can we predict the remaining useful life of an asset within several given time horizons (e.g. 30, 90, 180 days)?

In section 2., we'll use the events data to augment the assets table and gather as much insights as possible about questions 1-4, and others that might arise during analysis.

These insights will be used to guide question 5, in section 3.
There, we'll build and describe a functional pipeline to predict approximate remaining useful life for each asset.
**This pipeline can then be used in production, informing management decisions and guiding operation and maintenance teams in the field for reducing costs and downtime, and increase team productivity and customer satisfaction.**


# 2. Data analysis

## 2.1. Transforming features

### 2.1.1. Transforming categorical features

For computing the correlation of the different features, we have to transform some of them.
Specifically, we'll need to transform categorical features (team, line, material).

Let's make a copy of the original assets table and apply this transformation.

In [ ]:
assets2 = assets.copy()
events2 = events.copy()

In [ ]:
assets2 = pd.get_dummies(assets2, columns=['asset_material', 'asset_line', 'asset_weather_cluster', 'asset_install_team', 'asset_weather_cluster'])
events2 = pd.get_dummies(events2, columns=['type', 'planned'])

In [ ]:
assets2

### 2.1.2. Describing `previous_repairs` and `previous_unplanned`

The `previous_repairs` and `previous_unplanned` columns are present both in the events and assets tables.

In [ ]:
events[events['asset_id'] == 'A:xoauw0'] .sort_values('event_date')

`events`

After studying the events sequence for several assets, the following was concluded.
- In the events table, the previous_repairs column contains how many repair events ocurred since the last replacement event.
- The previous_unplanned contains how many unplanned repair events occurred snce the last replacement event.
- Each time a replacement happens, both counters are reset to 0.

In [ ]:
assets[assets['start_previous_unplanned'] > 0].head()

In [ ]:
asset_id = 'A:z7x72w'

In [ ]:
assets[assets['asset_id'] == asset_id]

In [ ]:
events[events['asset_id'] == asset_id] .sort_values('event_date')

`assets`
After studying the events sequence for several assets and their corresponding assets row, the following was concluded:
- The previous_repairs and previous_unplanned features of the assets table represen the total number repairs and number of unplanned repairs since the last replacement, at the moment of installation.
- This can be observed, for example, in the data regarding the asset with ID "A:z7x72w".
  - previous_repairs is 0 and previous_unplanned is 6 in the assets table. The first event is an unplanned repair. Both counters are incremented in the events table.
- It should be noted that in the events table, previous_unplanned is always lower than previous_repairs, but in the assets table such is not the case.

### 2.1.3 Transforming dates and time periods

We'll also need to transform features containing dates and time durations, because datetimes and timedeltas can't be directly correlated with other numeric data.

- We'll make a feature from the year to try to understand if older installations are less reliable.
- We'll use the month and day to make a sinusoidal wave with a period of one year and no phase shift, to understand if the seasons and time of the year also has an influence.*
  - We use a sinusoid so that the last days from one year are similar to the first days from the following year.
- We'll also add a weekday categorical variable (one hot encoded) to see understand if more 

\* [method for convertion](https://math.stackexchange.com/a/650235)

In [ ]:
def day_of_year_to_sin(day: pd.Timestamp,
                       period: float = 365.0,
                       phase_shift: float = 84.0):
    '''
    day: a pandas timestamp
    period: specifies the period of the wave
    phase_shift: shifts the sine wave so the the peaks are at specific days, 84 makes the peak ~match the solstice
    '''
    return math.sin((2 * math.pi) / period * (day.day_of_year - phase_shift))
    

weekdays = {0: 'monday',
            1: 'tuesday',
            2: 'wednesday',
            3: 'thursday',
            4: 'friday',
            5: 'saturday',
            6: 'sunday'
           }

def augment_datetime(df: pd.DataFrame, columns: List[str]):
    '''
    for each pandas.Timestamp column in dataframe df, create 3 features:
    - year (integer)
    - sine wave (from day and month) with yearly period
    - weekday, one hot encoded
    for each pandas.Timedelta column, convert to integer in days
    
    :returns: the input with 
    '''
    df_copy = df.copy()
    for col in columns:
        if col not in df_copy:
            raise Exception(f'column {col} not present in dataframe')
            
        if isinstance(df_copy[col].iloc[0], pd.Timestamp):
            df_copy[col + '_year'] = df_copy[col].map(lambda d: d.year)
            df_copy[col + '_weekday'] = df_copy[col].map(lambda d: weekdays[d.day_of_week])
            df_copy[col + '_sin_year'] = df_copy[col].map(day_of_year_to_sin)
            
            df_copy = pd.get_dummies(df_copy, columns=[col + '_weekday'])
            
        elif isinstance(df[col].iloc[0], pd.Timedelta):
            df_copy[col + '_days_int'] = df_copy[col].map(lambda td: td.days)
            
        else:
            raise TypeError(f'{col} is not of type pandas.Timestamp or pandas.Timedelta')
            
    return df_copy

In [ ]:
assets2 = augment_datetime(assets2, ['end_date', 'start_date', 'total_useful_life'])
events2 = augment_datetime(events2, ['event_date', 'installed_date', 'time_since_last_event'])

## 2.2. Data augmentation

Let's augment the assets table with events statistics.

In [ ]:
# create columns with default value 0, because of the 6 assets that don't have reported events
assets2['total_repairs'] = 0
assets2['total_replacements'] = 0

assets2['total_unplanned_repairs'] = 0
assets2['total_unplanned_replacements'] = 0

assets2['average_time_between_events'] = 0
assets2['std_time_between_events'] = 0

assets2 = assets2.set_index('asset_id')

events2['unplanned_repairs'] = events2['type_repair'] * events2['planned_False']
events2['unplanned_replacements'] = events2['type_replacement'] * events2['planned_False']


asset_group = events2.groupby('asset_id')

assets2['total_repairs'] = asset_group['type_repair'].sum()
assets2['total_replacements'] = asset_group['type_replacement'].sum()
assets2['average_time_between_events'] = asset_group['time_since_last_event_days_int'].mean()
assets2['std_time_between_events'] = asset_group['time_since_last_event_days_int'].std()


assets2['total_unplanned_repairs'] = asset_group['unplanned_repairs'].sum()
assets2['total_unplanned_replacements'] = asset_group['unplanned_replacements'].sum()

events2.drop(columns=['unplanned_repairs', 'unplanned_replacements'])

assets2[['total_repairs','total_replacements',
         'average_time_between_events',
         'std_time_between_events',
         'total_unplanned_repairs',
         'total_unplanned_replacements']]

## 2.3. Results and interpretation

In [ ]:
def corr_filter(df: pd.DataFrame,
                column: str,
                threshold: float = 0.2) -> pd.Series:
    corr = df.corr()[column]
    return corr[(corr <= -threshold) | (corr >= threshold)].sort_values()

### 2.3.1. How is the data distributed?

#### 2.3.1.1. Events types

The vast majority of events are unplanned $85\%$, for both repairs and replacements ($33\%$ and $52$, respectively).
The number of replacements ($57\%$) is slightly higher than that of repairs ($43\%$).

In [ ]:
events.groupby(['type', 'planned']).count()['asset_id'] / events.shape[0] * 100

In [ ]:
events.groupby(['type', 'planned']).count()['asset_id'].plot(kind='bar')

plt.title('Type of event')
plt.ylabel('# events')

#### 2.3.1.2. Asset line, weather, material

`weather clusters`
- A "standard" weather cluster accounts for most of the assets ($46\%$), but for the remaining clusters the assets are fairly well distributed.
- Useful life for heavy rain is lower than for the other weather clusters, and highest for sun.
However, the usage for heavy rain is highest and lowest for sun.
This means we cannot conclude right away that it's a particular weather cluster that is having an influence on useful life.

`asset lines`
- Assets' distribution over asset lines is almost uniform.
- Useful life is slightly higher for east and south lines.
Again, these lines have a lower usage.

`asset materials`
- Assets' distribution over materials is close to uniform.
- Steel has a slightly lower useful life, but is one of the materials with highest usage, so it makes sense.
- Iron has a high useful life, even though it has a high usage, which is an interesting result.
- Material variations for both usage and useful life are lower when compared to other variables.
For this reason, high confidence conclusions should not be drawn. 


In [ ]:
assets['total_useful_life'] = assets['total_useful_life'].map(lambda x: x.days)

In [ ]:
mean_useful_life = assets['total_useful_life'].mean()
print(f'average useful life: {mean_useful_life}')

In [ ]:
assets.groupby('asset_weather_cluster')['asset_id'].count().plot(kind='pie', autopct='%1.f%%')
plt.title('Assets by weather cluster')
plt.ylabel('')

In [ ]:
assets.groupby('asset_weather_cluster')['asset_trains_per_hour'].mean().plot(kind='bar')
plt.title('average usage per weather cluster')
plt.ylabel('hours')

In [ ]:
assets.groupby('asset_weather_cluster')['total_useful_life'].mean().plot(kind='bar')
plt.title('average useful life per weather cluster')
plt.ylabel('days')

In [ ]:
assets.groupby('asset_line')['asset_id'].count().plot(kind='pie', autopct='%1.f%%')
plt.title('Assets by line')
plt.ylabel('')

In [ ]:
assets.groupby('asset_line')['asset_trains_per_hour'].mean().plot(kind='bar')
plt.title('average usage per asset line')
plt.ylabel('hours')

In [ ]:
assets.groupby('asset_line')['total_useful_life'].mean().plot(kind='bar')
plt.title('average useful life per weather cluster')
plt.ylabel('days')

In [ ]:
assets.groupby('asset_material')['asset_id'].count().plot(kind='pie', autopct='%1.f%%')
plt.title('Assets by line')
plt.ylabel('')

In [ ]:
assets.groupby('asset_material')['asset_trains_per_hour'].mean().plot(kind='bar')
plt.title('average usage per asset material')
plt.ylabel('hours')

In [ ]:
assets.groupby('asset_material')['total_useful_life'].mean().plot(kind='bar')
plt.title('average useful life per asset material')
plt.ylabel('days')

#### 2.3.1.3. Asset usage

Since asset usage is distributed over a small set of values, we can plot this variable against others and try to understand some patterns. Asset usage was already studied above, so we'll analyze other relationships in these section.

- Surprisingly, useful life doesn't decrease linearly with increasing asset usage (inverse relationship).
This suggests that useful life has several different contributing factors.
- The number of maintenance events is higher for higher usage, which would be expected.


In [ ]:
assets.groupby('asset_trains_per_hour')['total_useful_life'].mean().plot(kind='bar')
plt.title('average useful life per asset usage')
plt.ylabel('days')

In [ ]:
assets2.groupby('asset_trains_per_hour')['total_repairs'].mean().plot(kind='bar')
plt.title('repairs per asset usage')
plt.ylabel('# events')

In [ ]:
assets2.groupby('asset_trains_per_hour')['total_replacements'].mean().plot(kind='bar')
plt.title('replacements per asset usage')
plt.ylabel('# events')

### 2.3.2. What influences total useful life?

In [ ]:
corr_filter(assets2, 'total_useful_life_days_int', threshold=0.2)

- There is very week correlation between useful life and most other variables.
- start_date has a strong correlation, but this likely means that the assets that were installed earlier have a better change to be represented longer in the events table.
- The number of events (repair and replacements) is also strongly correlated.
Again, an asset that is active for a long time is more likely to have more maintenance incidents, so this is to be expected.

### 2.3.3. What influences occurence of maintenance events?

In [ ]:
corr_filter(assets2, 'total_repairs', threshold=0.2)

In [ ]:
corr_filter(assets2, 'total_replacements', threshold=0.2)

In [ ]:
corr_filter(assets2, 'total_unplanned_repairs', threshold=0.2)

In [ ]:
corr_filter(assets2, 'total_unplanned_replacements', threshold=0.2)

In [ ]:
corr_filter(assets2, 'total_unplanned_repairs', threshold=0.2)

In [ ]:
events2.corr()['type_repair'].sort_values()

### 2.3.4. Team performance

**Assumption: the team that installed a particular asset is also the team that performed all repairs and replacements for that same asset throughout its useful life.**

- Assets are well distributed over teams (close to uniform).
- Events are well distributed over teams.
Teams with more assets have more events.
- The percentage of unplanned events is very close for all teams.
- There is little variability within useful life, among teams.
Of note, team 2 has a slightly lower average asset useful life, even though it has the lowest average asset usage, which would be more surprising, if values varied significantly.
- The conclusion is that the effectiveness and relative competence of teams is the same.

In [ ]:
assets2['team'] = assets.set_index('asset_id')['asset_install_team']
assets2.head()

In [ ]:
assets2.groupby('team')['total_useful_life'].count().plot(kind='pie', autopct='%1.f%%')
plt.title('Asset distribution by team')

In [ ]:
events3 = pd.merge(events2, assets2, how='inner', on=['asset_id'])

In [ ]:
events3.groupby('team')['event_id'].count().plot(kind='pie', autopct='%1.f%%')
plt.title('Events distribution by team')
plt.ylabel('')

In [ ]:
(events3.groupby('team')['planned_False'].sum() / events3.groupby('team')['planned_False'].count() * 100).plot(kind='bar')
plt.title('% of unplanned events by team')
plt.ylabel('%')

In [ ]:
assets.groupby('asset_install_team')['total_useful_life'].mean().plot(kind='bar')
plt.title('average useful life per team')
plt.ylabel('days')
plt.xlabel('team')

In [ ]:
assets.groupby('asset_install_team')['asset_trains_per_hour'].mean().plot(kind='bar')
plt.title('asset usage per team')
plt.ylabel('hours')
plt.xlabel('team')

### 2.3.5. Asset usage and event occurence

In [ ]:
(events3['event_date'].iloc[0] - events3['installed_date'].iloc[0]).days * 24 * events3['asset_trains_per_hour'].iloc[0]

In [ ]:
events3['trains_since_installed'] = (events3['event_date'] - events3['installed_date']).map(lambda x: x.days) * 24 * events3['asset_trains_per_hour']

In [ ]:
events3.corr()['type_repair']

In [ ]:
events3['trains_since_installed'].hist()

In [ ]:
events3[events3['type_replacement'] == 1]['trains_since_installed'].hist()

In [ ]:
events3[events3['type_replacement'] == 0]['trains_since_installed'].hist()

In [ ]:
events3[events3['planned_False'] == 1]['trains_since_installed'].hist()

In [ ]:
events3[events3['planned_False'] == 0]['trains_since_installed'].hist()

# 3. Predictive maintenance pipeline

The main goal with the predictive model is to reduce the number of unplanned events, since those drastically increase cost, downtime and liability.
Since most ot the events are unplanned ($85\%$), we have a **class imbalance**.
We'll make an assumption that will allow the simplification of the model.

**Assumptions**
- Models must predict occurence of events, the technical team will inspect the asset and determine one of three alternatives:
  - false positive (in time, the feedback data can be used to modify and further train the model)
  - repair necessary
  - replacement necessary


**Strategy**
- We can have different models, one for each time horizon and they'll be binary class models.
- Or, we can have a multi class model, where each time horizon is a different class.

**Plan**
- Augment events dataset
  - with data from assets table.
  - to indicate number of planned repair events in last 30/90/180/360 days
  - to indicate number of unplanned repair events in last 30/90/180/360 days
  - to indicate number of planned replacement events in last 30/90/180/360 days
  - to indicate number of unplanned replacement events in last 30/90/180/360 days
  - multiply usage per hour by the total number of hours
  - current life (event_date - start_date)
  - difference between average useful life and current life (average useful life - current_life)
    - average useful life must be updated when more data is collected
    - upgrade to use average useful life for events that match the features of the event being trained/inferred (e.g. weather, usage, team, material etc.)
- Add columns for indicating whether another event will happen in different time horizons: 30, 90, 180, 360 days - this is the target variable

## 3.1. Prepare dataset

As it is, the dataset is not ideal for creating a model.
There are not many variables with high correlation.
Moreover, we only have data for when the event already occurred, that is 

The dataset can be extended to include entries with dates between consequent events.
For example, consider that an event occurred in the 1st of march and then again in the 1st of may.
We can create 30 data points covering the month of january, with the indication that no event will happen in the next 30 days, but an event will happen in the next 90 days.
We can also create 30 data points for the month of february, with the indication that an event will happen in the next 30 days.

With this strategy, we'll be able to build a dataset that will contain a lot more rows than the events table.
To be specific, since the average useful life is 1530 days and we have 200 assets, we'll be able to build a dataset in the order of 300 000 rows.



In [ ]:
events3['time_since_last_event'].describe()

Features:
 - days since start_date - this will be a proxy for date (must be derived)
 - days since last installed date (must be derived)
 - trains since last installed date (must be derived)
 - previous repairs since last replacement (original data)
 - previous unplanned repairs since last replacement (original data)
 - number of previous replacements (must be derived)

Target
 - event in next 30 days (must be derived)
 - event in next 90 days (must be derived)
 - event in next 180 days (must be derived)
 - event in next 360 days (must be derived) (this might not make sense, since 75% of events happen within less than 181 days of each other)
 

### 3.1.1. Dataset creation

In [ ]:
def event_in_X_time(date: pd.Timestamp,
                    asset_data: pd.DataFrame,
                    horizon: pd.Timedelta = pd.Timedelta(days=30)
                   ) -> bool:
    return (asset_data['event_date'] < (date + horizon)).sum() > 0

def get_last_event(date: pd.Timestamp,
                   asset_data: pd.DataFrame) -> pd.Series:
    prev_events = asset_data[asset_data['event_date'] < date]
    if prev_events.shape[0] == 0:
        return None
    else:
        return prev_events.iloc[-1]
    
    
EventCounts = namedtuple('EventCounts', ['planned_repairs', 'unplanned_repairs', 'planned_replacements', 'unplanned_replacements'])
def get_past_events_count(date: pd.Timestamp,
                          asset_data: pd.DataFrame) -> EventCounts:
    prev_events = asset_data[asset_data['event_date'] < date]
    planned_repairs = (prev_events['type_repair'] * prev_events['planned_True']).sum()
    unplanned_repairs = (prev_events['type_repair'] * prev_events['planned_False']).sum()
    planned_replacements = (prev_events['type_replacement'] * prev_events['planned_True']).sum()
    unplanned_replacements = (prev_events['type_replacement'] * prev_events['planned_False']).sum()

    return EventCounts(planned_repairs=planned_repairs,
                       unplanned_repairs=unplanned_repairs,
                       planned_replacements=planned_replacements,
                       unplanned_replacements=unplanned_replacements
                      )
    
def build_dataset(asset_data: pd.DataFrame,
                  horizons: List[int] = [30, 60, 180],
                  static_columns: List[str] = [],
                  start_date: Optional[pd.Timestamp] = None,
                  end_date: Optional[pd.Timestamp] = None) -> pd.DataFrame:
    df = pd.DataFrame()
    
    # static_data
    asset_start_date = asset_data.iloc[0]['start_date']
    trains_per_hour = asset_data.iloc[0]['asset_trains_per_hour']
    
    # create date range
    first_day = asset_start_date if start_date is None else start_date
    last_day = asset_data.iloc[-1]['event_date'] if end_date is None else end_date
    df['date'] = pd.date_range(start=first_day,
                               end=last_day)
    
    df['asset_id'] = asset_data.iloc[0]['asset_id']
    
    ## loop dates instead of using map for performance
    ## (not enough data to justify efficient parallelization)
    
    # create columns
    df['days_since_asset_start'] = 0
    df['days_since_last_event'] = 0
    
    df['repairs_since_replacement'] = 0
    df['unplanned_repairs_since_replacement'] = 0
    
    df['unplanned_repairs'] = 0
    df['planned_repairs'] = 0
    df['unplanned_replacements'] = 0
    df['planned_replacements'] = 0
    
    
    for horizon_days in horizons:
        df[f'horizon_{horizon_days}'] = 0
        
    for i, d in df['date'].items():
        # print(d)
        last_event = get_last_event(d, asset_data)
        
        # days since asset start date
        df.at[i, 'days_since_asset_start'] = (d - asset_start_date).days

        # days since last event
        df.at[i, 'days_since_last_event'] = (d - last_event['event_date']).days if last_event is not None else 0
     
        # previous repairs since last replacement
        df.at[i, 'repairs_since_replacement'] = last_event['previous_repairs'] if last_event is not None else 0
    
        # previous unplanned repairs since last replacement
        df.at[i, 'unplanned_repairs_since_replacement'] = last_event['previous_unplanned'] if last_event is not None else 0
        
        
        event_counts = get_past_events_count(d, asset_data)
        # total previous unplanned repairs
        df.at[i, 'unplanned_repairs'] = event_counts.unplanned_repairs
        
        # total previous planned repairs
        df.at[i, 'planned_repairs'] = event_counts.planned_repairs
        
        # total previous unplanned replacements
        df.at[i, 'unplanned_replacements'] = event_counts.unplanned_replacements
        
        # total previous planned replacements
        df.at[i, 'planned_replacements'] = event_counts.planned_replacements
        
        # check existence of events in specified horizons (days)
        for horizon_days in horizons:
            df.at[i, f'horizon_{horizon_days}'] = event_in_X_time(d, asset_data,
                                                                    pd.Timedelta(days=horizon_days))
     
    # trains since last event
    df['trains_since_last_event'] = df['days_since_last_event'] * 24 * trains_per_hour

    # trains since asset start date
    df['trains_since_asset_start'] = df['days_since_asset_start'] * 24 * trains_per_hour

    # add data from static columns
    for col in static_columns:
        if col not in asset_data:
            raise ValueError(f'column {col} does not exist in asset data')
        df[col] = asset_data.iloc[0][col]
        
    return df

def get_asset_data(asset_id, events_data) -> pd.DataFrame:
    return events_data[events_data['asset_id'] == asset_id]

In [ ]:
# dataset cration parameteres
params = {
    'horizons': [30, 60, 90, 180, 360],
    'static_columns': []
}

#### 3.1.1.1. Create and store

In [ ]:
%%time
unique_ids = events['asset_id'].unique()

# first asset
asset_data = get_asset_data(unique_ids[0], events3)
df = build_dataset(asset_data, **params)

# remaing assets
for i, asset_id in enumerate(unique_ids[1:2]):
    print(i, asset_id, datetime.datetime.now())
    asset_data = get_asset_data(asset_id, events3)
    d = build_dataset(asset_data, **params)
    df = pd.concat([df, d], axis=0, ignore_index=True)

In [ ]:
static_df = assets[['asset_id', 'asset_install_team', 'asset_line', 'asset_material', 'asset_weather_cluster', 'end_date',]]

In [ ]:
df = pd.merge(df, static_df, on='asset_id')

In [ ]:
#remaining useful life
df['rul'] = (df['end_date'] - df['date']).map(lambda d: d.days)

In [ ]:
df.to_pickle(f'created_data_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")}.pkl')

#### 3.1.1.2. Load previous creation

In [ ]:
df = pd.read_pickle('created_data_2022_05_02_22_50.pkl')

### 3.1.2. Pre-processing

In [ ]:
for horizon in params['horizons']:
    df[f'horizon_{horizon}'] = df[f'horizon_{horizon}'].map(lambda x: 1 if x else 0)
    
    # measure class imbalance
    pct = df[f'horizon_{horizon}'].sum() / df.shape[0]
    print(f'class horizon {horizon}: {pct*100}% positive')

In [ ]:
df['horizon_30'].hist()

We have a high class imbalance.
Let's down sample the dataset to have a balanced class distribution.
We only have about 25000 samples that don't have an event in the future 30 days.
Let's downsample the one that have to that amount too.

In [ ]:
def binary_balanced_sample(x: pd.DataFrame, target: str, tolerance: float = 0.1) -> Tuple[pd.DataFrame, pd.DataFrame]:
    '''
    :param x: input dataset
    :type x: pandas.DataFrame
    :param target: column name of binary class
    :type target: str
    :returns: balanced dataset, 
    '''
    unique_values = x[target].unique()
    if unique_values.size != 2:
        raise TypeError(f'{target} has {unique_values.size} classes, not 2')
    
    idx = [x[x[target] == c].index for c in unique_values]
    small_class_idx, big_class_idx = sorted(idx, key=lambda k: k.size)
    
    # check if already balanced
    
    # balance by sampling from the big class the number of rows in the small class
    big_sampled = x.iloc[big_class_idx].sample(small_class_idx.size)
    x_balanced = pd.concat([big_sampled, x.iloc[small_class_idx]])
    left_out = x.iloc[x.index.difference(x_balanced.index)]
    
    return x_balanced, left_out

In [ ]:
df_balanced, left_out = binary_balanced_sample(df, 'horizon_30')

In [ ]:
df_balanced.corr()

1. **dimensionaly reduction?**
2. **which features?**
3. **k-folds?**
4. **which model?**
5. **grid/random search?**




### 3.1.3. Data split and feature selection

In [ ]:
dfX = df
print(dfX.shape)

Since we have a decently sized dataset, a simple train/validation split should suffice to get decent results, i.e. k-fold is not necessary.

In [ ]:
dfX.columns

In [ ]:
columns_target = ['horizon_30', 'horizon_60', 'horizon_90', 'horizon_180', 'horizon_360']

columns_infer = [
    'days_since_asset_start',
    'days_since_last_event',
    'previous_repairs',
    'previous_unplanned',
    'trains_since_last_event',
    'trains_since_asset_start'
]


def getXy(data: pd.DataFrame, train_cols: List[str], target: str) -> Tuple[pd.DataFrame, pd.Series]:
    return data[train_cols], data[target]


In [ ]:
target_col = columns_target[0]

# balance chosen dataset according to specified target column
df_balanced, left_out = binary_balanced_sample(dfX, target_col)

# split into "inference" set and target set
X, y = getXy(df_balanced, columns_infer, target_col)

# split left out set into validation inference/target sets
Xval, yval = getXy(left_out, columns_infer, target_col)

# train/test split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

print(f"{df_balanced.shape[0]} \t rows in balanced datset")
print(f"{Xtrain.shape[0]} \t rows in train set")
print(f"{Xtest.shape[0]} \t rows in test set")
print(f"{Xval.shape[0]} \t rows in validation set")

## 3.2. Single Class Multi Model

In [ ]:
def compute_metrics(ytrue: np.array, ypred: np.array) -> dict:
    acc = metrics.accuracy_score(ytrue, ypred)
    recall = metrics.recall_score(ytrue, ypred)
    confusion = confusion_matrix(ytrue, ypred)
    return {'accuracy': acc, 'recall': recall, 'confusion': confusion}

### 3.2.1 Decision Tree

In [ ]:
model_meta = {
    'type': 'DecisionTreeClassifier',
    'params': {
        'max_depth': 1,
        'criterion': 'entropy'
    }
}

with mlflow.start_run(run_name=f"{model_meta['type']}_{target_col}"):
    dt = DecisionTreeClassifier(**model_meta['params'])
    dt.fit(Xtrain, ytrain)

    ytrain_pred = dt.predict(Xtrain)
    ytest_pred = dt.predict(Xtest)
    yval_pred = dt.predict(Xval)
    
    run_metrics = {}
    run_metrics['train'] = compute_metrics(ytrain, ytrain_pred)
    run_metrics['test'] = compute_metrics(ytest, ytest_pred)
    run_metrics['val'] = compute_metrics(yval, yval_pred)
    
    ConfusionMatrixDisplay(run_metrics['test']['confusion']).plot()
    plt.title('Confusion matrix for test set')
    figCT = plt.gcf()

    ConfusionMatrixDisplay(run_metrics['val']['confusion']).plot()
    plt.title('Confusion matrix for validation set')
    figCV = plt.gcf()
    
    figFI = plt.figure()
    plt.barh(list(X.columns), dt.feature_importances_)
    plt.title('Feature importance')
    plt.xlabel('Gini importance')
    plt.ylabel('Feature')
    plt.tight_layout()

    figDP = plt.figure()
    plt.title('Decision path')
    tree.plot_tree(dt)
    
    # log figures
    mlflow.log_figure(figCT, "test_confusion.png")
    mlflow.log_figure(figCV, "val_confusion.png")
    mlflow.log_figure(figFI, "feature_importance.png")
    mlflow.log_figure(fig, "decision_path.png")

    # log datasets size
    mlflow.log_param('n_features', Xtrain.shape[1])
    mlflow.log_param('train_size', Xtrain.shape[0])
    mlflow.log_param('test_size', Xtest.shape[0])
    mlflow.log_param('val_size', Xval.shape[0])
    
    # log params
    for param, val in model_meta['params'].items():   
        mlflow.log_param(param, val)
    
    # log metrics
    for s in ['train', 'test', 'val']:
        for m in ['accuracy', 'recall']:
            print(f"{s}_{m} - {run_metrics[s][m]}")
            mlflow.log_metric(f"{s}_{m}", run_metrics[s][m])

            
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
    # Model registry does not work with file store
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(dt, "model", registered_model_name="DecisionTree_" + target_col)
    else:
        mlflow.sklearn.log_model(dt, "model")

### 3.2.2. Random Forest

In [ ]:
model_meta = {
    'type': 'RandomForestClassifier',
    'params': {
        'max_depth': None,
        'n_estimators': 10,
        'criterion': 'gini'
    }
}

with mlflow.start_run(run_name=f"{model_meta['type']}_{target_col}"):
    dt = RandomForestClassifier(**model_meta['params'])
    dt.fit(Xtrain, ytrain)

    ytrain_pred = dt.predict(Xtrain)
    ytest_pred = dt.predict(Xtest)
    yval_pred = dt.predict(Xval)
    
    run_metrics = {}
    run_metrics['train'] = compute_metrics(ytrain, ytrain_pred)
    run_metrics['test'] = compute_metrics(ytest, ytest_pred)
    run_metrics['val'] = compute_metrics(yval, yval_pred)
    
    ConfusionMatrixDisplay(run_metrics['test']['confusion']).plot()
    plt.title('Confusion matrix for test set')
    figCT = plt.gcf()

    ConfusionMatrixDisplay(run_metrics['val']['confusion']).plot()
    plt.title('Confusion matrix for validation set')
    figCV = plt.gcf()
    
    figFI = plt.figure()
    plt.barh(list(X.columns), dt.feature_importances_)
    plt.title('Feature importance')
    plt.xlabel('Gini importance')
    plt.ylabel('Feature')
    plt.tight_layout()

    
    # log figures
    mlflow.log_figure(figCT, "test_confusion.png")
    mlflow.log_figure(figCV, "val_confusion.png")
    mlflow.log_figure(figFI, "feature_importance.png")
   
    
    # log params
    for param, val in model_meta['params'].items():   
        mlflow.log_param(param, val)
        
    for s in ['train', 'test', 'val']:
        for m in ['accuracy', 'recall']:
            print(f"{s}_{m} - {run_metrics[s][m]}")
            mlflow.log_metric(f"{s}_{m}", run_metrics[s][m])

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(dt, "model", registered_model_name="DecisionTree_" + target_col)
    else:
        mlflow.sklearn.log_model(dt, "model")

## 3.3. Multi Class Single Model

## 3.4. Predict Remaining Useful Life (RUL) (Regression)

The Remaining Useful Life is the amount of time left until the retirement of an asset.

**Assumption: the retirement of an asset, here, is the end_date in the assets table.**

At any given day, the retirement of an asset is that day subtracted from the end date.

In the first models, we tried to predict whether a maintenance event would occur withing a given time frame.
Here, we'll try to predict the RUL of a given asset, given what we know about it know.
Some of this knowledge is static, e.g. material, line, weather cluster, team.
Other is derived, e.g. how many trains passed since the beginning, how many repairs, how many replacements.
These derived features have already been computed during the dataset creation, in section 3.1.1.

In [ ]:
dfX = df.copy()

In [ ]:
m = LinearRegression()

In [ ]:
xx = dfX[['trains_since_asset_start', 'rul']].sort_values('trains_since_asset_start')
x = xx['trains_since_asset_start'].to_numpy()
y = xx['rul'].to_numpy()

In [ ]:
plt.scatter(x[::10],y[::10])
plt.ylabel('RUL')
plt.xlabel('trains_since_asset_start')
plt.title('RUL/Trains since asset start')

In [ ]:
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

# let us train the model

m.fit(x, y)

In [ ]:
ŷ = m.predict(x)
metrics.mean_absolute_error(y, ŷ)